#Very Important Spark Learning - BY LEARNING this PROGRAM - WE BECOME A DATA ENGINEER (DATA CURATION DEVELOPER & DATA ANALYST)

next level of SQL (Spark SQL) + Python Function based programming (Framework of Spark DSL) + Datawarehouse (Datalake+Lakehouse) -> Transformation & Analytics

##**1. Data Munging** - (Cleanup) Process of transforming and mapping data from Raw form into Tidy(usable) format with the intent of making it more appropriate and valuable for a variety of downstream purposes such for further Transformation/Enrichment, Egress/Outbound, analytics, Datascience/AI application & Reporting

**Passive Data Munging** - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns. <br>
    
**Active Data Munging**
1. Combining Data + Schema Evolution/Merging/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

![Stage1](stage1.png)

###a. Passive Data Munging - 
- Visible - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns.

####1. Visibily/Manually opening the file we found couple of data patterns (Manual Exploratory Data Analysis)
- It is a Structured data with comma seperator (CSV)
- No Header, No comments, footer is there in the data
- Total columns are (seperator + 1)
- Data Quality
- - Null columns & null rows are there
- - duplicate rows & Duplicate keys
- - format issues are there (age is not in number format eg. 7-7)
- - Uniformity issues (Artist, artist)
- - Number of columns are more or less than the expected
- eg. 4000011,Francis,McNamara,47,Therapist,NewYork & 4000014,Beth,Woodard,65
- - Identification of data type

####2. Programatically lets try to find couple of data patterns applying EDA - passively (without modifying, just for description)

In [0]:
#We have to define Spark Session to enter into Spark application
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.appName("we47_Bread_n_Butter2_Important_Application").getOrCreate()

In [0]:
#Data Exploration programatically
rawdf1=spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custsmodified",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1.show(20,False)
display(rawdf1.take(20))

In [0]:
#Important passive Munging - EDA of schema/structure functions we can use
rawdf1.printSchema()#I am realizing the id & age columns are having some non numerical values (supposed to be numeric)
print(rawdf1.columns)#I am understanding the column numbers/order and the column names
print(rawdf1.dtypes)#Realizing the datatype of every columns (even we can do programattic column & type identification for dynamic programming)
print(rawdf1.schema)#To identify the structure of the data in the StructType and StructField format

In [0]:
#Important passive Datamunging - EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - Standard deviation tells you how much the data varies from the average (mean).
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
print("actual count of the data",rawdf1.count())
print("de-duplicated record (all columns) count",rawdf1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given id column count",rawdf1.dropDuplicates(['id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf1.describe())
display(rawdf1.summary())

###**b. Active Data Munging**
1. Combining Data + Schema Evolution/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

In [0]:
#Questions related to multiple files/paths/sub path handling
#I have data in different filenames in a single/multiple location, i need to read all these data in a df - path=["path1/file1","path1/file2","path2/file3"]
#I have data in single pattern of file names in a single/multiple locations or subfolders, i need to read all these data in a df - path=["path1/","path1/","path2/"], pathGlobFilter=\"custsm*\", recursiveFileLookup=True

#Questions related handling evolving data structure with data ingested in different days/periods - Ans. Schema Evolution

#Evolution is growth over the time (Filesystem level).. Eg. Source is sending data with additional columns week over week in csv format
#1. Read and write in Serialized format( ORC,Parquet)
#2. Read DF with mergeSchema = True

#Questions related handling data from different sources with different related structure in a same day - Ans. Schema Merging/Melting (Dataframe level).. Eg. Source1 is sending custsmodified_NY with 5 columns and Source2 is sending custsmodified_TX with 4 columns
#1. Read file1 in DF1, read file2 in DF2
#2. Create DF3 by merging DF1 and DF2 using df1.unionByName(df2,allowMissingColumns=True)

####1. Combining Data + Schema Evolution/Merging (Structuring) - Preliminary Datamunging

In [0]:
#Extraction (Ingestion) methodologies\n",
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custsmodified")
#2. Multiple files (with different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custsmodified","/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custsmodifiedsample.txt"])
#3. Multiple files in multiple paths or sub paths
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/","/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1"],recursiveFileLookup=True,pathGlobFilter="custsm*")